<a href="https://colab.research.google.com/github/monroews/4590FSP/blob/master/Hydraulics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Chemicals

Authors: Bridget Childs, Ananya Gangadhar, Lok Yee Jacqueline Wong



# Task Definition

Write a paragraph describing what is included in your task.

# Executive Summary 

Highlight dominant constraints and your recommendation for the design.


# Constraints

* Relevant regulations
* client expectations
* site and budget
* process capabilities including process design equations
* ease of use 
* operation requirements
* maintainence requirements

# Physics

Briefly describe the physics that will determine whether your portion of the design succeeds or fails. Present relevant equations describing those physical laws and explain how you will use those concepts to guide your design.

Replace this with equations that are important to your part of the design. Explain what each equation means.

$$ F = ma$$

where $F$ is the force, $m$ is the mass, and $a$ is the acceleration.

# Table listing potential strategies to meet the constraints and treatment objectives

Please remove my two rows of examples!

| strategy | advantages  | disadvantages | data needed for decision |
|---|---|---|---|
| physics-based design | high potential for success | requires knowledge | lots |
| wishful design | easy design steps | likely won't work | none |

# Table listing vendors and contact information

See this tab delimited [spreadsheet](https://raw.githubusercontent.com/monroews/4590FSP/master/vendors.txt).
Add rows to the vendor spreadsheet as you find them. The sheet can easily be edited in Excel. Just make sure to save it as a tab delimited sheet. 

# Major Decisions

Write several paragraphs describing the major decisions that you will need to make in order to create a design and outline a strategy for how you will obtain the necessary information to make those decisions.

# Next Steps and Challenges

Summarize the next steps that you will be taking to move the design forward. Identify any challenges where you may need assistance.

In [1]:
!pip install aguaclara
!pip install molmass

     |████████████████████████████████| 92kB 2.6MB/s 
     |████████████████████████████████| 163kB 8.2MB/s 
  Created wheel for aguaclara: filename=aguaclara-0.2.0-cp36-none-any.whl size=99623 sha256=8429f6352a11d414120ed2280471fad3b14488c241871f9ad5a7d5eb396b67f0
  Stored in directory: /root/.cache/pip/wheels/34/e9/34/0d3722243083a982623290c205c14a08b54ed290ee29219fd2
  Created wheel for pint: filename=Pint-0.8.1-py2.py3-none-any.whl size=134156 sha256=9deb7d77b2d056f2e877266fc2e17edc5c43f2322a3e82928617115925952e70
  Stored in directory: /root/.cache/pip/wheels/b4/56/36/7c4d6a4e3660e233b95e3b0482551915cb184abca0f93ee1dd
Successfully built aguaclara pint
     |████████████████████████████████| 71kB 2.3MB/s 


In [0]:
from aguaclara.core.units import unit_registry as u
import aguaclara as ac
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import molmass as mol
!wget https://raw.githubusercontent.com/monroews/4590FSP/master/Fairmont.py
import Fairmont as fmt

In [9]:
print('The average flow to be treated at Fairmont is',fmt.q_ave.to(u.Mgal/u.day),'or')
print('The maximum flow to be treated at Fairmont is',fmt.q_max.to(u.Mgal/u.day))

The average flow to be treated at Fairmont is 206.8 megagallon / day
The maximum flow to be treated at Fairmont is 480 megagallon / day


In [12]:
vendors = pd.read_csv('https://raw.githubusercontent.com/monroews/4590FSP/master/vendors.txt',sep='\t')
vendors.head()

,item description,vendor,contact first name,contact last name,vendor website,product info
0,plate settlers,MRI,nan,nan,http://www.meurerresearch.com/,http://www.meurerresearch.com/literaturemedia/...
